In [1]:
import pandas as pd
import os

In [2]:
dir = "Folder_A/"
file_list = os.listdir(dir)
file_list

['E-10000232.xlsx',
 'E-10000957.xlsx',
 'E-12345634.xlsx',
 'E-12345654.xlsx',
 'E-12345678.xlsx',
 'E-12345910.xlsx',
 '~$E-10000232.xlsx']

In [3]:
file_list = [file for file in file_list if "~$" not in file] # do not consider opened files (~$ prefix for windows)

file_list

['E-10000232.xlsx',
 'E-10000957.xlsx',
 'E-12345634.xlsx',
 'E-12345654.xlsx',
 'E-12345678.xlsx',
 'E-12345910.xlsx']

In [4]:
# row numb per file and total

row_total = 0

for file in file_list:
    
    print(
        len(
            pd.read_excel(dir+file, skiprows=6, index_col=None))) # delet first 6 rows
    
    row_total += len(pd.read_excel(dir+file, skiprows=6, index_col=None))

row_total

578
578
112
112
112
112


1604

* all raw files in one excel sheet
* add a column to track file source


In [5]:
# OPTION A

df_A = pd.DataFrame()

#read and concat files
for i in range(len(file_list)):
    path = dir + file_list[i]
    append_df = pd.read_excel(path, skiprows=6, index_col=None) # delet first 6 rows
    append_df["File Name"] = file #to track file name
    df_A = pd.concat([df_A, append_df], ignore_index=True, axis=0)

In [6]:
# OPTION B

df = pd.DataFrame()

# read and concat files
for file in file_list:
    append_df = pd.read_excel(dir+file, skiprows=6, index_col=None) # delet first 6 rows
    append_df["File Name"] = file #to track file name
    df = df.append(append_df)

In [7]:
df.shape

(1604, 43)

In [10]:
df.head()

,Opportunity ID,Opportunity Name,Project ID,Project Name,Engagement ID,Engagement Name,Client ID,Client Name,Week Ending Date,Transaction Date,...,Category Code Description,Expense Amount,Expense Description,Sub Category Description,Account Name/ID,Journal ID,Voucher ID,Vendor Name/ID,Transaction Type,File Name
0,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-014,Client1,E-10000232,engagement 1,123456789,client 1,2019-06-21,2019-06-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Labor,E-10000232.xlsx
1,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-014,Client1,E-10000232,engagement 1,123456789,client 1,2019-06-21,2019-06-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Labor,E-10000232.xlsx
2,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-014,Client1,E-10000232,engagement 1,123456789,client 1,2019-06-21,2019-06-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Labor,E-10000232.xlsx
3,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-014,Client1,E-10000232,engagement 1,123456789,client 1,2019-06-21,2019-06-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Labor,E-10000232.xlsx
4,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-014,Client1,E-10000232,engagement 1,123456789,client 1,2019-06-21,2019-06-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Labor,E-10000232.xlsx


Delete entire rows if the Column “Rank” contains “Unknown” or “Intern (CS)”

In [11]:
df = df[(df.Rank != "Unknown") & (df.Rank != "Intern (CS)")]
df.shape[0] # new row total

1538

Colour entire rows if the column “Activity Code Description” contains “General”

In [12]:
table_index = df.index

In [13]:
selected_index = df[df["Activity Code Description"] == 'General'].index
selected_index

Int64Index([  0,   1,   4,  14,  16,  18,  19,  20,  21,  22,
            ...
             31,  35,  36,  62,  84,  92,  95,  98, 102, 109],
           dtype='int64', length=486)

In [14]:
# PENDING , see documentation, pandas.style 

•	Create a separate sheet and name it as “Senior Rank”. 
<br>
•	Filter “Executive Director” & “Manager” data from the column “Rank” and copy paste filtered data to the sheet “Senior Rank”

In [15]:
df_SR = df[(df.Rank == "Executive Director") | (df.Rank == "Manager")]
df_SR.Rank.head()

3                Manager
11               Manager
12               Manager
18    Executive Director
19    Executive Director
Name: Rank, dtype: object

In [16]:
df_SR.shape

(304, 43)

In [17]:
# see last cell about the new excel sheet

Add sum to the columns “Charged Hours”, “NSR” & “ANSR” in consolidated sheet

In [18]:
# sum up numeric fields
row_sum = df.sum(numeric_only = True)
row_sum

Client ID                 1.345734e+11
Employee GUI              1.526301e+09
Grade                     2.788000e+03
Charged Hours             1.778600e+03
NSR                       3.690706e+05
NSR Rate                  3.182360e+05
Rate Card Rate            0.000000e+00
Rate Card Amount          0.000000e+00
ANSR                      2.380220e+05
EAF Reserve Allocation    1.310486e+05
Direct Cost               1.518942e+05
Cost Rate                 1.299286e+05
Expense Amount            1.857200e+02
Journal ID                0.000000e+00
Voucher ID                0.000000e+00
Vendor Name/ID            0.000000e+00
dtype: float64

In [19]:
fields = ["Charged Hours", "NSR", "ANSR"]
last_row_added = df.shape[0]

for field in fields:
    df.loc[last_row_added, field] = row_sum[field]

df[fields].tail(10)

,Charged Hours,NSR,ANSR
103,2.0,278.0,165.332076
104,0.4,55.6,33.066415
105,0.5,69.5,41.333019
106,0.3,41.7,24.799811
107,0.2,27.8,16.533208
108,1.2,360.0,214.099092
109,1.0,113.0,67.203326
110,NaN,NaN,0.000000
111,NaN,NaN,0.000000
1538,1778.6,369070.6,238022.022752


In [20]:
df.iloc[df.shape[0]-1][fields] # last row

Charged Hours           1778.6
NSR                   369070.6
ANSR             238022.022752
Name: 1538, dtype: object

In [21]:
df.iloc[(df.shape[0]-1), 0] = "Total"
df.tail(3)

,Opportunity ID,Opportunity Name,Project ID,Project Name,Engagement ID,Engagement Name,Client ID,Client Name,Week Ending Date,Transaction Date,...,Category Code Description,Expense Amount,Expense Description,Sub Category Description,Account Name/ID,Journal ID,Voucher ID,Vendor Name/ID,Transaction Type,File Name
110,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-019,Project Name,E-12345678,AZ_GCR TAC 2019/2020,12346.0,Client Name,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E-12345910.xlsx
111,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-019,Project Name,E-12345678,AZ_GCR TAC 2019/2020,12346.0,Client Name,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E-12345910.xlsx
1538,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Identify the last row in Consolidated file and add a comment “Consolidation done” in the cell of last row +1 

In [22]:
import numpy as np


df.loc[df.shape[0]] = np.nan # add extra row
df.loc[df.shape[0]] = np.nan # add extra row again

df.iloc[df.shape[0]-1, 
            0] = "Consolidation done"

df.tail()

,Opportunity ID,Opportunity Name,Project ID,Project Name,Engagement ID,Engagement Name,Client ID,Client Name,Week Ending Date,Transaction Date,...,Category Code Description,Expense Amount,Expense Description,Sub Category Description,Account Name/ID,Journal ID,Voucher ID,Vendor Name/ID,Transaction Type,File Name
110,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-019,Project Name,E-12345678,AZ_GCR TAC 2019/2020,12346.0,Client Name,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E-12345910.xlsx
111,O-195750,AZ_GCR TR/BTC + TAC + FS + VAT + TP 2016,P-195750-019,Project Name,E-12345678,AZ_GCR TAC 2019/2020,12346.0,Client Name,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E-12345910.xlsx
1538,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1540,Consolidation done,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.iloc[df.shape[0]-1, 3]

nan

Remove "null" from new rows

In [24]:
df.iloc[[df.shape[0]-2,df.shape[0]-1], :]


,Opportunity ID,Opportunity Name,Project ID,Project Name,Engagement ID,Engagement Name,Client ID,Client Name,Week Ending Date,Transaction Date,...,Category Code Description,Expense Amount,Expense Description,Sub Category Description,Account Name/ID,Journal ID,Voucher ID,Vendor Name/ID,Transaction Type,File Name
1539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1540,Consolidation done,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Export file

In [25]:
with pd.ExcelWriter(dir+'Consolidated.xlsx') as writer:  
    df.to_excel(writer, sheet_name='Consolidated files', index=False)
    df_SR.to_excel(writer, sheet_name='Senior Rank', index=False)